In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from scipy.misc import comb as comb1
from scipy.special import gamma
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import scipy.linalg as la

# Problem 10.17

Return to the problem about the city in which there have been 496 female
births and 489 male births in the past year. We assume the number of female
births is binomially distributed with parameter $\theta$, and we are interested in
comparing two possible models: $M_0$ is the model where $\theta$ = 1/2 and $M_1$, is
a model where $\theta$ can take any value in [0, 1].

(i) Compute $p(D | M)$ for each model, assuming a prior for $\theta$ under $M_1$ of
$p(\theta | M_1) = B(1, 1)$.

In [2]:
nfemale = 496
nmale = 489
# p(D|M_i)
dataprob_model0 = comb1(nfemale+nmale, nfemale)*(.5)**(nfemale+nmale)

$p(D|M_1) 
= \frac{p(D|\Theta, M_1) P(\Theta|M_1)}{P(\Theta|D,M_1)} 
= \frac{{985 \choose 496}\Theta^{496} (1-\Theta)^{489} \Theta^0(1-\Theta)^0 \frac{\Gamma(2)}{\Gamma(1)\Gamma(1)}}{\Theta^{1+496-1}(1-\Theta)^{1+489-1} \frac{\Gamma(2+985)}{\Gamma(1+496)\Gamma(1+489)}} $

Thus, we see that 
$ p(D|M_1) 
= {985 \choose 496} \frac{\Gamma(497)\Gamma(490)}{\Gamma(987)}
= \frac{985! 496! 489!}{496! 489! 986!} 
= \frac{1}{986} $
because all of the terms with $\Theta$ cancel out and $\frac{\Gamma(2)}{\Gamma(1)\Gamma(1)} = 1$

In [3]:
#logdataprob_model1 = np.log(comb(nfemale+nmale, nfemale)) + np.log(gamma(497)) + np.log(gamma(490)) - np.log(gamma(987))
dataprob_model1 = 1/986.

In [4]:
print('P(D|M_0) = {}\nP(D|M_1) = {}'.format(dataprob_model0, dataprob_model1))

P(D|M_0) = 0.0247925009971
P(D|M_1) = 0.00101419878296


(ii) Assuming a prior of $p(M_i)=1/2$ for both models, compute $p(M | D)$
for each model. By this measure which model is preferred?

$ P(M_i|D) 
= \frac{P(D|M_i) .5}{P(D|M_0).5 + P(D|M_1).5} 
= \frac{P(D|M_i)}{P(D|M_0) + P(D|M_1)}$

In [5]:
total_dataprob = dataprob_model0 + dataprob_model1
model0prob = dataprob_model0 / total_dataprob
model1prob = dataprob_model1 / total_dataprob
print("P(M_0|D) = {}\nP(M_1|D) = {}".format(model0prob, model1prob))

P(M_0|D) = 0.960700175086
P(M_1|D) = 0.0392998249139


# Problem 10.18

Fit a linear model to the wages.csv dataset using scikit-learn instead of
statsmodels. To do this use the commands:

from sklearn import linear_model

regr = linear_model.LinearRegression()

X = df[['female','educ','exper','tenure','married','female*married','numdep','nonwhite']]

y = df['wage']

regr.fit(X,y)

Compare the calculated coefficients regr.coef_ with the coefficients computed
with statsmodels.

In [6]:
df = pd.read_csv('wages.csv')
df['female*married'] = df['female']*df['married']
regr = linear_model.LinearRegression()
X = df[['female','educ','exper','tenure','married','female*married','numdep','nonwhite']]
y = df['wage']
regr.fit(X,y)
regr.coef_

array([-0.34413527,  0.56308905,  0.02091295,  0.12976128,  1.73557801,
       -2.3578156 ,  0.08909389, -0.2142343 ])

In [7]:
X = sm.add_constant(X)
print(sm.OLS(y,X).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     41.90
Date:                Fri, 08 Dec 2017   Prob (F-statistic):           1.42e-51
Time:                        17:53:48   Log-Likelihood:                -1301.6
No. Observations:                 526   AIC:                             2621.
Df Residuals:                     517   BIC:                             2660.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
const             -2.5664      0.780     -3.

Thus the coefficients are the same.

# Problem 10.19

Using sklearn’s test-train tools create a training set that is 70% of the full
data and a test set that is 30% of the full data. Fit both a full model (all
the features listed above) and a model having only the features ['educ','
tenure','married','female*married'] on the training data and compare
the Mean Square Error (MSE = score in scikit-learn) for each model.

In [8]:
train, test = train_test_split(df, test_size=.3, train_size = .7)

regr_full_model = linear_model.LinearRegression()
X1 = train[['female','educ','exper','tenure','married','female*married','numdep','nonwhite']]
y1 = train['wage']
regr_full_model.fit(X1,y1)

testX1 = test[['female','educ','exper','tenure','married','female*married','numdep','nonwhite']]
testy = test['wage']
predict1 = np.dot(testX1, regr_full_model.coef_) + regr_full_model.intercept_
mse1 = mean_squared_error(testy, predict1)


regr_restricted_model = linear_model.LinearRegression()
X2 = train[['educ','tenure','married','female*married']]
regr_restricted_model.fit(X2,y1)

testX2 = test[['educ','tenure','married','female*married']]
predict2 = np.dot(testX2, regr_restricted_model.coef_) + regr_restricted_model.intercept_
mse2 = mean_squared_error(testy, predict2)

print("MSE for Full Model:\t\t{}\nMSE for Restricted Model:\t{}".format(mse1,mse2))

MSE for Full Model:		8.01842306049
MSE for Restricted Model:	7.98223145876


# Problem 10.20

Use scikit-learn to compare each of the two previous models with a 7-fold
cross validation.

In [9]:
regr_full_model = linear_model.LinearRegression()
X = df[['female','educ','exper','tenure','married','female*married','numdep','nonwhite']]
y = df['wage']
cv_score1 = -1*cross_val_score(regr_full_model, X,y, scoring='neg_mean_squared_error', cv=7)

regr_restricted_model = linear_model.LinearRegression()
X_restricted = df[['educ','tenure','married','female*married']]
cv_score2 = -1*cross_val_score(regr_restricted_model, X_restricted, y, scoring='neg_mean_squared_error', cv=7)

In [10]:
print("Full Model MSE:\t\t{}\nRestricted Model MSE:\t{}".format(cv_score1, cv_score2))
print("\n\nFull Model MSE Normed:\t\t{}\nRestricted Model MSE Normed:\t{}".format(la.norm(cv_score1), la.norm(cv_score2)))

Full Model MSE:		[ 15.04807859   7.80036641   8.23095008   9.70621285   4.45192344
   7.8667286    8.0232463 ]
Restricted Model MSE:	[ 14.95477276   7.37446954   8.30851754   9.84279749   4.61098581
   7.41039223   7.89874299]


Full Model MSE Normed:		24.3993240648
Restricted Model MSE Normed:	24.1351486303


Above, we take the 2-norm of the vector of mean squared errors to compare the two models. We see that the restricted model does a bit better, which makes sense because it would have less of a problem with overfitting the training data set.